In [7]:
# Load required libraries
library(Seurat)
library(dplyr)
library(ggplot2)
library(cowplot)   # For combining plots
library(ggrepel)

# Set working directory (adjust as needed)
setwd("/Users/katherineridley/Projects/CosMx/APP/Cortex Results")

# Load your main Seurat object (adjust filename if needed)
seurat_obj <- readRDS("combined_seurat_C_plaquedistance.RDS")

# Check the distribution of cell types
print("Celltypes distribution in the main Seurat object:")
print(table(seurat_obj@meta.data$Celltypes, useNA = "ifany"))

# Get unique cell types (excluding NA)
cell_types_unique <- unique(seurat_obj@meta.data$Celltypes)
cell_types_unique <- cell_types_unique[!is.na(cell_types_unique)]
print("Unique cell types to process:")
print(cell_types_unique)

# Loop through each cell type
for (ct in cell_types_unique) {
  message("Processing cell type: ", ct)
  
  # Subset the Seurat object by cell type using WhichCells() for safety
  cells_to_use <- WhichCells(seurat_obj, expression = Celltypes == ct)
  
  if (length(cells_to_use) == 0) {
    message("  Skipping ", ct, " - no cells found.")
    next
  }
  
  sub_obj <- subset(seurat_obj, cells = cells_to_use)
  
  # Run standard subclustering workflow on the subset
  # (Assumes variable features have been identified; if not, you may run FindVariableFeatures)
  sub_obj <- RunPCA(sub_obj, features = VariableFeatures(sub_obj))
  sub_obj <- FindNeighbors(sub_obj, dims = 1:10)
  sub_obj <- FindClusters(sub_obj, resolution = 0.5)
  sub_obj <- RunUMAP(sub_obj, dims = 1:10)
  
  # Save the cluster identities as a new metadata column ("Subcluster")
  sub_obj$Subcluster <- Idents(sub_obj)
  
  # Plot and save UMAP with cluster labels for visual inspection
  umap_plot <- DimPlot(sub_obj, reduction = "umap", label = TRUE, repel = TRUE) +
    ggtitle(paste("Subcluster UMAP for", ct))
  umap_file <- paste0("UMAP_Subcluster_", gsub(" ", "_", ct), ".png")
  ggsave(filename = umap_file, plot = umap_plot, width = 8, height = 6)
  message("  Saved UMAP plot for ", ct, " in ", umap_file)
  
  # Run FindAllMarkers to find genes that characterize each cluster
  markers <- FindAllMarkers(sub_obj, only.pos = TRUE, min.pct = 0.25)
  
  # Optionally, print a quick summary table of top markers per cluster for inspection
  clusters_summary <- markers %>% group_by(cluster) %>% top_n(3, wt = avg_log2FC)
  message("  Top markers per cluster for ", ct, ":")
  print(clusters_summary)
  
  # Save the marker genes as a CSV for this cell type
  markers_csv <- paste0("markers_Subclusters_", gsub(" ", "_", ct), ".csv")
  write.csv(markers, file = markers_csv, row.names = FALSE)
  message("  Saved markers CSV for ", ct, " as ", markers_csv)
  
  # (Optional) You might save the subclustered object itself for future review:
  sub_obj_file <- paste0("subclustered_", gsub(" ", "_", ct), ".rds")
  saveRDS(sub_obj, file = sub_obj_file)
  message("  Saved subclustered Seurat object for ", ct, " as ", sub_obj_file)
}

message("Subclustering and marker CSV generation complete.")


[1] "Celltypes distribution in the main Seurat object:"

        Astrocytes  Endothelial cells Excitatory Neurons Inhibitory Neurons 
             11829               6599             130652               8192 
       Macrophages          Microglia               OPCs   Oligodendrocytes 
               187                805               1319              26260 
              <NA> 
              6723 
[1] "Unique cell types to process:"
[1] "Excitatory Neurons" "Oligodendrocytes"   "Astrocytes"        
[4] "Microglia"          "Endothelial cells"  "Inhibitory Neurons"
[7] "OPCs"               "Macrophages"       


Processing cell type: Excitatory Neurons

Warning message:
"Removing 6723 cells missing data for vars requested"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating Seurat objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating Seurat objects"
PC_ 1 
Positive: 

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 130652
Number of edges: 3858222

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.8881
Number of communities: 17
Elapsed time: 43 seconds


12:59:05 UMAP embedding parameters a = 0.9922 b = 1.112

12:59:05 Read 130652 rows and found 10 numeric columns

12:59:05 Using Annoy for neighbor search, n_neighbors = 30

12:59:05 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

12:59:15 Writing NN index file to temp file /var/folders/c7/rvxz8xg96pxfhyt34649yfqh0000gn/T//RtmprXYgVm/filef50b449ad66b

12:59:15 Searching Annoy index using 1 thread, search_k = 3000

12:59:53 Annoy recall = 100%

12:59:54 Commencing smooth kNN distance calibration using 1 thread
 with target n_neighbors = 30

12:59:57 Initializing from normalized Laplacian + noise (using RSpectra)

12:59:59 Commencing optimization for 200 epochs, with 5487048 positive edges

13:00:39 Optimization finished

Rasterizing points since number of points exceeds 

# A tibble: 51 x 7
# Groups:   cluster [17]
   p_val avg_log2FC pct.1 pct.2 p_val_adj cluster gene   
   <dbl>      <dbl> <dbl> <dbl>     <dbl> <fct>   <chr>  
 1     0      1.01  0.657 0.329         0 0       Pcp4   
 2     0      0.935 0.363 0.186         0 0       Pde1a  
 3     0      0.818 0.326 0.177         0 0       Slc8a1 
 4     0      2.28  0.868 0.313         0 1       Pcp4   
 5     0      1.44  0.702 0.29          0 1       Pak1   
 6     0      1.54  0.502 0.175         0 1       Pde1a  
 7     0      4.94  0.975 0.102         0 2       Gfap   
 8     0      3.28  0.315 0.084         0 2       Vim    
 9     0      1.58  0.361 0.21          0 2       Mt1    
10     0      1.72  0.487 0.15          0 3       Rasgrf2
# i 41 more rows


...

  Saved markers CSV for OPCs as markers_Subclusters_OPCs.csv

  Saved subclustered Seurat object for OPCs as subclustered_OPCs.rds

Processing cell type: Macrophages

Warning message:
"Removing 6723 cells missing data for vars requested"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating Seurat objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating 

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 187
Number of edges: 6185

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.6370
Number of communities: 2
Elapsed time: 0 seconds


13:47:42 UMAP embedding parameters a = 0.9922 b = 1.112

13:47:42 Read 187 rows and found 10 numeric columns

13:47:42 Using Annoy for neighbor search, n_neighbors = 30

13:47:42 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

13:47:42 Writing NN index file to temp file /var/folders/c7/rvxz8xg96pxfhyt34649yfqh0000gn/T//RtmprXYgVm/filef50b7ae580f0

13:47:42 Searching Annoy index using 1 thread, search_k = 3000

13:47:42 Annoy recall = 100%

13:47:43 Commencing smooth kNN distance calibration using 1 thread
 with target n_neighbors = 30

13:47:44 Initializing from normalized Laplacian + noise (using RSpectra)

13:47:44 Commencing optimization for 500 epochs, with 7024 positive edges

13:47:44 Optimization finished

  Saved UMAP plot for Macrophages in UMAP_Subcluster_Mac

# A tibble: 6 x 7
# Groups:   cluster [2]
     p_val avg_log2FC pct.1 pct.2 p_val_adj cluster gene 
     <dbl>      <dbl> <dbl> <dbl>     <dbl> <fct>   <chr>
1 1.38e- 5       2.54 0.321 0.053  1.31e- 2 0       Braf 
2 4.87e- 4       2.08 0.268 0.067  4.63e- 1 0       Ngf  
3 8.02e- 4       2.16 0.375 0.16   7.62e- 1 0       Vtn  
4 1.60e-20       3.12 0.947 0.571  1.52e-17 1       Cd74 
5 3.65e-11       2.63 0.64  0.214  3.47e- 8 1       H2-Aa
6 6.87e- 9       3.47 0.627 0.295  6.53e- 6 1       Spp1 


  Saved markers CSV for Macrophages as markers_Subclusters_Macrophages.csv

  Saved subclustered Seurat object for Macrophages as subclustered_Macrophages.rds

Subclustering and marker CSV generation complete.



In [8]:
# Load required libraries
library(Seurat)
library(dplyr)
library(ggplot2)
library(cowplot)   # For combining plots
library(ggrepel)

# Set working directory (adjust as needed)
setwd("/Users/katherineridley/Projects/CosMx/APP/Hpc Results")

# Load your main Seurat object (adjust filename if needed)
seurat_obj <- readRDS("combined_seurat_H_plaquedistance.RDS")

# Check the distribution of cell types
print("Celltypes distribution in the main Seurat object:")
print(table(seurat_obj@meta.data$Celltypes, useNA = "ifany"))

# Get unique cell types (excluding NA)
cell_types_unique <- unique(seurat_obj@meta.data$Celltypes)
cell_types_unique <- cell_types_unique[!is.na(cell_types_unique)]
print("Unique cell types to process:")
print(cell_types_unique)

# Loop through each cell type
for (ct in cell_types_unique) {
  message("Processing cell type: ", ct)
  
  # Subset the Seurat object by cell type using WhichCells() for safety
  cells_to_use <- WhichCells(seurat_obj, expression = Celltypes == ct)
  
  if (length(cells_to_use) == 0) {
    message("  Skipping ", ct, " - no cells found.")
    next
  }
  
  sub_obj <- subset(seurat_obj, cells = cells_to_use)
  
  # Run standard subclustering workflow on the subset
  # (Assumes variable features have been identified; if not, you may run FindVariableFeatures)
  sub_obj <- RunPCA(sub_obj, features = VariableFeatures(sub_obj))
  sub_obj <- FindNeighbors(sub_obj, dims = 1:10)
  sub_obj <- FindClusters(sub_obj, resolution = 0.5)
  sub_obj <- RunUMAP(sub_obj, dims = 1:10)
  
  # Save the cluster identities as a new metadata column ("Subcluster")
  sub_obj$Subcluster <- Idents(sub_obj)
  
  # Plot and save UMAP with cluster labels for visual inspection
  umap_plot <- DimPlot(sub_obj, reduction = "umap", label = TRUE, repel = TRUE) +
    ggtitle(paste("Subcluster UMAP for", ct))
  umap_file <- paste0("UMAP_Subcluster_", gsub(" ", "_", ct), ".png")
  ggsave(filename = umap_file, plot = umap_plot, width = 8, height = 6)
  message("  Saved UMAP plot for ", ct, " in ", umap_file)
  
  # Run FindAllMarkers to find genes that characterize each cluster
  markers <- FindAllMarkers(sub_obj, only.pos = TRUE, min.pct = 0.25)
  
  # Optionally, print a quick summary table of top markers per cluster for inspection
  clusters_summary <- markers %>% group_by(cluster) %>% top_n(3, wt = avg_log2FC)
  message("  Top markers per cluster for ", ct, ":")
  print(clusters_summary)
  
  # Save the marker genes as a CSV for this cell type
  markers_csv <- paste0("markers_Subclusters_", gsub(" ", "_", ct), ".csv")
  write.csv(markers, file = markers_csv, row.names = FALSE)
  message("  Saved markers CSV for ", ct, " as ", markers_csv)
  
  # (Optional) You might save the subclustered object itself for future review:
  sub_obj_file <- paste0("subclustered_", gsub(" ", "_", ct), ".rds")
  saveRDS(sub_obj, file = sub_obj_file)
  message("  Saved subclustered Seurat object for ", ct, " as ", sub_obj_file)
}

message("Subclustering and marker CSV generation complete.")


[1] "Celltypes distribution in the main Seurat object:"

        Astrocytes  Endothelial cells Excitatory Neurons Inhibitory Neurons 
             11311               4742              73903               4028 
       Macrophages          Microglia               OPCs   Oligodendrocytes 
               189                698                759              12743 
              <NA> 
              5139 
[1] "Unique cell types to process:"
[1] "Excitatory Neurons" "Oligodendrocytes"   "Astrocytes"        
[4] "Endothelial cells"  "Inhibitory Neurons" "OPCs"              
[7] "Macrophages"        "Microglia"         


Processing cell type: Excitatory Neurons

Warning message:
"Removing 5139 cells missing data for vars requested"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating Seurat objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating Seurat objects"
PC_ 1 
Positive: 

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 73903
Number of edges: 2259611

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9059
Number of communities: 15
Elapsed time: 22 seconds


13:56:13 UMAP embedding parameters a = 0.9922 b = 1.112

13:56:13 Read 73903 rows and found 10 numeric columns

13:56:13 Using Annoy for neighbor search, n_neighbors = 30

13:56:13 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

13:56:18 Writing NN index file to temp file /var/folders/c7/rvxz8xg96pxfhyt34649yfqh0000gn/T//RtmprXYgVm/filef50b67980637

13:56:18 Searching Annoy index using 1 thread, search_k = 3000

13:56:40 Annoy recall = 100%

13:56:41 Commencing smooth kNN distance calibration using 1 thread
 with target n_neighbors = 30

13:56:42 Initializing from normalized Laplacian + noise (using RSpectra)

13:56:44 Commencing optimization for 200 epochs, with 3155288 positive edges

13:57:06 Optimization finished

  Saved UMAP plot for Excitatory Neurons in UMAP_Su

# A tibble: 45 x 7
# Groups:   cluster [15]
   p_val avg_log2FC pct.1 pct.2 p_val_adj cluster gene  
   <dbl>      <dbl> <dbl> <dbl>     <dbl> <fct>   <chr> 
 1     0      2.95  0.67  0.099         0 0       Jun   
 2     0      2.72  0.615 0.092         0 0       Tiam1 
 3     0      2.80  0.39  0.052         0 0       Sema5a
 4     0      0.967 0.766 0.407         0 1       Aldoc 
 5     0      0.929 0.992 0.656         0 1       Apoe  
 6     0      0.916 0.923 0.589         0 1       Slc1a2
 7     0      1.93  0.585 0.189         0 2       Gap43 
 8     0      1.56  0.428 0.148         0 2       Sstr4 
 9     0      1.56  0.362 0.123         0 2       Pde1a 
10     0      2.51  0.844 0.2           0 3       Stmn2 
# i 35 more rows


...

  Saved markers CSV for OPCs as markers_Subclusters_OPCs.csv

  Saved subclustered Seurat object for OPCs as subclustered_OPCs.rds

Processing cell type: Macrophages

Warning message:
"Removing 5139 cells missing data for vars requested"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating Seurat objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating 

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 189
Number of edges: 5747

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.6778
Number of communities: 4
Elapsed time: 0 seconds


14:34:52 UMAP embedding parameters a = 0.9922 b = 1.112

14:34:52 Read 189 rows and found 10 numeric columns

14:34:52 Using Annoy for neighbor search, n_neighbors = 30

14:34:52 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

14:34:52 Writing NN index file to temp file /var/folders/c7/rvxz8xg96pxfhyt34649yfqh0000gn/T//RtmprXYgVm/filef50b24087e85

14:34:52 Searching Annoy index using 1 thread, search_k = 3000

14:34:52 Annoy recall = 100%

14:34:52 Commencing smooth kNN distance calibration using 1 thread
 with target n_neighbors = 30

14:34:53 Initializing from normalized Laplacian + noise (using RSpectra)

14:34:53 Commencing optimization for 500 epochs, with 6906 positive edges

14:34:53 Optimization finished

  Saved UMAP plot for Macrophages in UMAP_Subcluster_Mac

# A tibble: 12 x 7
# Groups:   cluster [4]
      p_val avg_log2FC pct.1 pct.2 p_val_adj cluster gene  
      <dbl>      <dbl> <dbl> <dbl>     <dbl> <fct>   <chr> 
 1 4.33e- 4       1.97 0.288 0.095  4.11e- 1 0       Tiam1 
 2 2.37e- 3       1.74 0.329 0.155  1   e+ 0 0       Fos   
 3 6.36e- 3       1.76 0.315 0.164  1   e+ 0 0       Nrsn1 
 4 1.13e-16       2.99 0.906 0.448  1.07e-13 1       Cd74  
 5 8.89e-14       2.90 0.656 0.152  8.44e-11 1       Cst7  
 6 5.85e- 5       2.88 0.531 0.296  5.56e- 2 1       Spp1  
 7 2.92e- 7       1.76 0.794 0.419  2.77e- 4 2       P2ry12
 8 3.08e- 4       1.90 0.353 0.11   2.93e- 1 2       Snca  
 9 5.16e- 3       2.01 0.294 0.116  1   e+ 0 2       Rgs5  
10 9.29e-16       5.30 0.944 0.205  8.83e-13 3       Ptgds 
11 4.88e-14       4.12 0.889 0.187  4.64e-11 3       Apod  
12 1.61e-12       3.21 0.833 0.17   1.53e- 9 3       Ptn   


  Saved markers CSV for Macrophages as markers_Subclusters_Macrophages.csv



In [1]:
# Load required libraries
library(Seurat)
library(dplyr)
setwd("/Users/katherineridley/Projects/CosMx/APP/Cortex Results")
# Define file paths
full_obj_file <- "/Users/katherineridley/Projects/CosMx/APP/Cortex Results/combined_seurat_C_plaquedistance.RDS"
subcluster_dir <- "/Users/katherineridley/Projects/CosMx/APP/Cortex Results"

# Load the full Seurat object
combined_seurat <- readRDS(full_obj_file)

# For later merging refined annotations, get a copy of the metadata
full_meta <- combined_seurat@meta.data

# Define the clusters to remove per cell type (using character values)
clusters_to_remove <- list(
  "Excitatory Neurons" = c("2", "4", "7", "10", "11", "12"),
  "Oligodendrocytes"    = c("0", "2", "4"),
  "Astrocytes"          = c("4", "7", "9"),
  "Endothelial cells"   = c("0"),
  "OPCs"                = c("1", "2", "5", "6"),
  "Inhibitory Neurons"  = character(0)  # No removal
)

# --- Step 1: Remove cells from the full object based on unwanted subclusters ---
# We'll loop over each cell type for which we have a corresponding subclustered file.
for (ct in names(clusters_to_remove)) {
  message("Processing removal for cell type: ", ct)
  
  # Construct filename for the subclustered object.
  file_name <- paste0("subclustered_", gsub(" ", "_", ct), ".rds")
  sub_obj_path <- file.path(subcluster_dir, file_name)
  if (!file.exists(sub_obj_path)) {
    message("  Subclustered file for ", ct, " not found: ", sub_obj_path, ". Skipping removal for this type.")
    next
  }
  
  # Load the subclustered Seurat object for this cell type.
  sub_obj <- readRDS(sub_obj_path)
  
  # Ensure that the subclustered object has a metadata column "Subcluster"
  if (!"Subcluster" %in% colnames(sub_obj@meta.data)) {
    message("  'Subcluster' column not found in ", ct, " subclustered object. Skipping removal.")
    next
  }
  
  # Identify cells (barcodes) to remove: those that in the subclustered object have a Subcluster in the removal list.
  rm_clusters <- clusters_to_remove[[ct]]
  if (length(rm_clusters) == 0) {
    message("  No clusters defined to remove for ", ct, ".")
    next
  }
  
  cells_to_remove <- rownames(sub_obj@meta.data)[sub_obj@meta.data$Subcluster %in% rm_clusters]
  message("  Removing ", length(cells_to_remove), " cells from ", ct, " based on subcluster criteria.")
  
  # In the full object, for cells of this cell type, remove those cells.
  # First, get cell barcodes in combined_seurat for this cell type:
  cells_full <- WhichCells(combined_seurat, expression = Celltypes == ct)
  # Intersection with cells_to_remove:
  cells_rm <- intersect(cells_full, cells_to_remove)
  message("  Total cells to remove for ", ct, ": ", length(cells_rm))
  
  if(length(cells_rm) > 0){
    combined_seurat <- subset(combined_seurat, cells = setdiff(Cells(combined_seurat), cells_rm))
  }
}

# --- Step 2: Create refined metadata columns for astrocytes (Celltypes_r) and inhibitory neurons (Celltypes_i) ---
# First, start by copying over the existing 'Celltypes' into new columns, so that non-target cell types remain unchanged.
combined_seurat@meta.data$Celltypes_r <- combined_seurat@meta.data$Celltypes  
combined_seurat@meta.data$Celltypes_i <- NA  # default NA for non-inhibitory cells

# For astrocytes:
# Load the astrocyte subclustered object
astro_file <- file.path(subcluster_dir, "subclustered_Astrocytes.rds")
if (file.exists(astro_file)) {
  astro_obj <- readRDS(astro_file)
  
  # We want only the cells that remain in the full object (after removal)
  astro_barcodes <- intersect(Cells(astro_obj), Cells(combined_seurat))
  if(length(astro_barcodes) > 0) {
    # For these astrocytes, take their Subcluster annotations.
    astro_subclusters <- astro_obj@meta.data$Subcluster[match(astro_barcodes, rownames(astro_obj@meta.data))]
    
    # Define refined astrocyte annotation: reactive if subcluster in c("2", "6", "8"), homeostatic otherwise.
    refined_astro <- ifelse(astro_subclusters %in% c("2", "6", "8"),
                            "Reactive Astrocytes",
                            "Homeostatic Astrocytes")
    
    # Place these refined labels in the new metadata column Celltypes_r for astrocytes.
    full_meta <- combined_seurat@meta.data  # current metadata from full object
    astro_idx <- which(full_meta$Celltypes == "Astrocytes" & rownames(full_meta) %in% astro_barcodes)
    full_meta$Celltypes_r[astro_idx] <- refined_astro
    combined_seurat@meta.data <- full_meta
  } else {
    message("  No astrocyte cells in the astro subclustered object are present in the full object.")
  }
} else {
  message("Astrocyte subclustered file not found. Astrocyte refinement skipped.")
}

# For inhibitory neurons:
# Load the inhibitory neuron subclustered object; assume its filename is "subclustered_Inhibitory_Neurons.rds"
inhib_file <- file.path(subcluster_dir, "subclustered_Inhibitory_Neurons.rds")
if (file.exists(inhib_file)) {
  inhib_obj <- readRDS(inhib_file)
  
  # Consider cells from inhib_obj that are still in combined_seurat.
  inhib_barcodes <- intersect(Cells(inhib_obj), Cells(combined_seurat))
  if(length(inhib_barcodes) > 0) {
    inhib_subclusters <- inhib_obj@meta.data$Subcluster[match(inhib_barcodes, rownames(inhib_obj@meta.data))]
    
    # Define refined inhibitory annotation:
    # Clusters 0, 1 -> "Parvalbumin Interneurons"
    # Cluster 3 -> "Somatostatin Interneurons"
    # Cluster 5 -> "VIP Interneurons"
    # Cluster 6 -> "CCK Interneurons"
    # All other inhib clusters -> "Interneurons (Other)"
    refined_inhib <- ifelse(inhib_subclusters %in% c("0", "1"), "Parvalbumin Interneurons",
                     ifelse(inhib_subclusters == "3", "Somatostatin Interneurons",
                     ifelse(inhib_subclusters == "5", "VIP Interneurons",
                     ifelse(inhib_subclusters == "6", "CCK Interneurons",
                            "Interneurons (Other)"))))
    
    # Place these labels in new metadata column Celltypes_i for inhibitory neurons
    full_meta <- combined_seurat@meta.data
    inhib_idx <- which(full_meta$Celltypes == "Inhibitory Neurons" & rownames(full_meta) %in% inhib_barcodes)
    full_meta$Celltypes_i[inhib_idx] <- refined_inhib
    combined_seurat@meta.data <- full_meta
  } else {
    message("  No inhibitory neuron cells in the subclustered object are present in the full object.")
  }
} else {
  message("Inhibitory neuron subclustered file not found. Inhibitory subtype refinement skipped.")
}

# For all other cell types, you can simply set:
# If Celltypes_r is NA, then assign original Celltypes.
full_meta <- combined_seurat@meta.data
full_meta$Celltypes_r[is.na(full_meta$Celltypes_r)] <- full_meta$Celltypes[is.na(full_meta$Celltypes_r)]
combined_seurat@meta.data <- full_meta

# Save the updated full Seurat object as a new RDS file.
saveRDS(combined_seurat, file = "combined_seurat_C_plaquedistance_cleanup.RDS")
message("Updated Seurat object saved as combined_seurat_plaquedistance_cleanup.RDS")


Loading required package: SeuratObject

Loading required package: sp




Attaching package: 'SeuratObject'


The following objects are masked from 'package:base':

    intersect, t



Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Processing removal for cell type: Excitatory Neurons

  Removing 46497 cells from Excitatory Neurons based on subcluster criteria.

Warning message:
"Removing 6723 cells missing data for vars requested"
  Total cells to remove for Excitatory Neurons: 46497

Warning message:
"Not validating FOV objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating Seurat obje

In [2]:
# Load required libraries
library(Seurat)
library(dplyr)
setwd("/Users/katherineridley/Projects/CosMx/APP/Hpc Results")
# Define file paths
full_obj_file <- "/Users/katherineridley/Projects/CosMx/APP/Hpc Results/combined_seurat_H_plaquedistance.RDS"
subcluster_dir <- "/Users/katherineridley/Projects/CosMx/APP/Hpc Results"

# Load the full Seurat object
combined_seurat <- readRDS(full_obj_file)

# For later merging refined annotations, get a copy of the metadata
full_meta <- combined_seurat@meta.data

# Define the clusters to remove per cell type (using character values)
clusters_to_remove <- list(
  "Excitatory Neurons" = c("6", "7", "8", "9", "10", "11"),
  "Oligodendrocytes"    = c("6", "8", "9"),
  "Astrocytes"          = c("5", "9"),
  "Endothelial cells"   = c("0"),
  "OPCs"                = c("2", "5"),
  "Inhibitory Neurons"  = c("6")
)

# --- Step 1: Remove cells from the full object based on unwanted subclusters ---
# We'll loop over each cell type for which we have a corresponding subclustered file.
for (ct in names(clusters_to_remove)) {
  message("Processing removal for cell type: ", ct)
  
  # Construct filename for the subclustered object.
  file_name <- paste0("subclustered_", gsub(" ", "_", ct), ".rds")
  sub_obj_path <- file.path(subcluster_dir, file_name)
  if (!file.exists(sub_obj_path)) {
    message("  Subclustered file for ", ct, " not found: ", sub_obj_path, ". Skipping removal for this type.")
    next
  }
  
  # Load the subclustered Seurat object for this cell type.
  sub_obj <- readRDS(sub_obj_path)
  
  # Ensure that the subclustered object has a metadata column "Subcluster"
  if (!"Subcluster" %in% colnames(sub_obj@meta.data)) {
    message("  'Subcluster' column not found in ", ct, " subclustered object. Skipping removal.")
    next
  }
  
  # Identify cells (barcodes) to remove: those that in the subclustered object have a Subcluster in the removal list.
  rm_clusters <- clusters_to_remove[[ct]]
  if (length(rm_clusters) == 0) {
    message("  No clusters defined to remove for ", ct, ".")
    next
  }
  
  cells_to_remove <- rownames(sub_obj@meta.data)[sub_obj@meta.data$Subcluster %in% rm_clusters]
  message("  Removing ", length(cells_to_remove), " cells from ", ct, " based on subcluster criteria.")
  
  # In the full object, for cells of this cell type, remove those cells.
  # First, get cell barcodes in combined_seurat for this cell type:
  cells_full <- WhichCells(combined_seurat, expression = Celltypes == ct)
  # Intersection with cells_to_remove:
  cells_rm <- intersect(cells_full, cells_to_remove)
  message("  Total cells to remove for ", ct, ": ", length(cells_rm))
  
  if(length(cells_rm) > 0){
    combined_seurat <- subset(combined_seurat, cells = setdiff(Cells(combined_seurat), cells_rm))
  }
}

# --- Step 2: Create refined metadata columns for astrocytes (Celltypes_r) and inhibitory neurons (Celltypes_i) ---
# First, start by copying over the existing 'Celltypes' into new columns, so that non-target cell types remain unchanged.
combined_seurat@meta.data$Celltypes_r <- combined_seurat@meta.data$Celltypes  
combined_seurat@meta.data$Celltypes_i <- NA  # default NA for non-inhibitory cells

# For astrocytes:
# Load the astrocyte subclustered object
astro_file <- file.path(subcluster_dir, "subclustered_Astrocytes.rds")
if (file.exists(astro_file)) {
  astro_obj <- readRDS(astro_file)
  
  # We want only the cells that remain in the full object (after removal)
  astro_barcodes <- intersect(Cells(astro_obj), Cells(combined_seurat))
  if(length(astro_barcodes) > 0) {
    # For these astrocytes, take their Subcluster annotations.
    astro_subclusters <- astro_obj@meta.data$Subcluster[match(astro_barcodes, rownames(astro_obj@meta.data))]
    
    # Define refined astrocyte annotation: reactive if subcluster in c("2", "6", "8"), homeostatic otherwise.
    refined_astro <- ifelse(astro_subclusters %in% c("0", "7", "8"),
                            "Reactive Astrocytes",
                            "Homeostatic Astrocytes")
    
    # Place these refined labels in the new metadata column Celltypes_r for astrocytes.
    full_meta <- combined_seurat@meta.data  # current metadata from full object
    astro_idx <- which(full_meta$Celltypes == "Astrocytes" & rownames(full_meta) %in% astro_barcodes)
    full_meta$Celltypes_r[astro_idx] <- refined_astro
    combined_seurat@meta.data <- full_meta
  } else {
    message("  No astrocyte cells in the astro subclustered object are present in the full object.")
  }
} else {
  message("Astrocyte subclustered file not found. Astrocyte refinement skipped.")
}

# For inhibitory neurons:
# Load the inhibitory neuron subclustered object; assume its filename is "subclustered_Inhibitory_Neurons.rds"
inhib_file <- file.path(subcluster_dir, "subclustered_Inhibitory_Neurons.rds")
if (file.exists(inhib_file)) {
  inhib_obj <- readRDS(inhib_file)
  
  # Consider cells from inhib_obj that are still in combined_seurat.
  inhib_barcodes <- intersect(Cells(inhib_obj), Cells(combined_seurat))
  if(length(inhib_barcodes) > 0) {
    inhib_subclusters <- inhib_obj@meta.data$Subcluster[match(inhib_barcodes, rownames(inhib_obj@meta.data))]
    
    # Define refined inhibitory annotation:
    # Clusters 0, 1 -> "Parvalbumin Interneurons"
    # Cluster 3 -> "Somatostatin Interneurons"
    # Cluster 5 -> "VIP Interneurons"
    # Cluster 6 -> "CCK Interneurons"
    # All other inhib clusters -> "Interneurons (Other)"
    refined_inhib <- ifelse(inhib_subclusters %in% "1", "Parvalbumin Interneurons",
                     ifelse(inhib_subclusters == c("4", "8"), "Somatostatin Interneurons",
                     ifelse(inhib_subclusters == "9", "VIP Interneurons",
                     ifelse(inhib_subclusters == "7", "CCK Interneurons",
                            "Interneurons (Other)"))))
    
    # Place these labels in new metadata column Celltypes_i for inhibitory neurons
    full_meta <- combined_seurat@meta.data
    inhib_idx <- which(full_meta$Celltypes == "Inhibitory Neurons" & rownames(full_meta) %in% inhib_barcodes)
    full_meta$Celltypes_i[inhib_idx] <- refined_inhib
    combined_seurat@meta.data <- full_meta
  } else {
    message("  No inhibitory neuron cells in the subclustered object are present in the full object.")
  }
} else {
  message("Inhibitory neuron subclustered file not found. Inhibitory subtype refinement skipped.")
}

# For all other cell types, you can simply set:
# If Celltypes_r is NA, then assign original Celltypes.
full_meta <- combined_seurat@meta.data
full_meta$Celltypes_r[is.na(full_meta$Celltypes_r)] <- full_meta$Celltypes[is.na(full_meta$Celltypes_r)]
combined_seurat@meta.data <- full_meta

# Save the updated full Seurat object as a new RDS file.
saveRDS(combined_seurat, file = "combined_seurat_H_plaquedistance_cleanup.RDS")
message("Updated Seurat object saved as combined_seurat_plaquedistance_cleanup.RDS")


Processing removal for cell type: Excitatory Neurons

  Removing 19703 cells from Excitatory Neurons based on subcluster criteria.

Warning message:
"Removing 5139 cells missing data for vars requested"
  Total cells to remove for Excitatory Neurons: 19703

Warning message:
"Not validating FOV objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating Seurat objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating FOV objects"
Warning message:
"Not validating Centroids objects"
Warning message:
"Not validating FOV objects"
Warning messa